In [ ]:
import sys
import os
dir = os.path.abspath('')
while not dir.endswith('ardt'): dir = os.path.dirname(dir)
if not dir in sys.path: sys.path.append(dir)

In [ ]:
import numpy as np

from datasets import load_dataset, load_from_disk
from huggingface_hub import login

from utils.helpers import find_root_dir

from access_tokens import HF_WRITE_TOKEN

In [ ]:
login(token=HF_WRITE_TOKEN)
ARDT_DIR = find_root_dir()

In [ ]:
TYPE = 'train'
VERSION = 'v4'

In [ ]:
dataset = load_from_disk(f'{ARDT_DIR}/datasets-raw/nonadv-raw-data/ppo_halfcheetah_v4_train_v4')

In [ ]:
total_max = 0
total_min = 0

for t in dataset:
    total_max = max(total_max, np.sum(t['rewards']))
    total_min = min(total_min, np.sum(t['rewards']))
    
print("Max total reward:", total_max)
print("Min total reward:", total_min)

In [ ]:
# Convert the 'actions' column to a numpy array
pr_actions = np.array(dataset['actions'])

# Create a new array for 'pr_actions', keeping the original values
new_pr_actions = pr_actions.astype(np.float32)

# Create a new array for 'adv_actions', filled with zeros
adv_actions = np.zeros_like(new_pr_actions)

# Convert back to lists if needed
new_pr_actions = new_pr_actions.tolist()
adv_actions = adv_actions.tolist()

# Update the dataset
dataset = dataset.add_column('pr_actions', new_pr_actions)
dataset = dataset.add_column('adv_actions', adv_actions)
dataset = dataset.remove_columns(['actions'])

In [ ]:
dataset.save_to_disk(f'{ARDT_DIR}/datasets/ppo_noadv_{TYPE}_halfcheetah_{VERSION}')